In [9]:
!pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26781 sha256=d352d7af102f4c30832d02c71f18327479a565bfa243481ccaa6a9f0a694e6f5
  Stored in directory: /Users/rahulnathan/Library/Caches/pip/wheels/63/e8/ec/75b1df467ecf57b6ececb32cb16f4e86697cbfe55cb0c51f07
Successfully built pandasql


In [1]:
import pandas as pd
from pandasql import sqldf

In [2]:
df_campaign = pd.read_csv("campaign_info.csv")
df_performance = pd.read_csv("marketing_performance.csv")
df_rev = pd.read_csv("website_revenue.csv")

In [3]:
df_campaign.head()

,id,name,status,last_updated_date
0,64441550,Campaign1,paused,2023-08-08 0:00:00
1,55304737,Campaign2,enabled,2023-07-25 0:00:00
2,89363211,Campaign3,enabled,2023-08-06 0:00:00
3,86363015,Campaign4,paused,2023-08-14 0:00:00
4,99058240,Campaign5,enabled,2023-07-28 0:00:00


In [4]:
df_performance.head()

,date,campaign_id,geo,cost,impressions,clicks,conversions
0,2023-07-29 0:00:00,64441550,United States-TX,107.86,2486,1737,294
1,2023-08-14 0:00:00,64441550,United States-OH,230.99,2465,1747,275
2,2023-08-12 0:00:00,64441550,United States-GA,214.94,2461,1306,352
3,2023-08-04 0:00:00,55304737,United States-NY,123.74,1069,1356,317
4,2023-08-19 0:00:00,86363015,United States-GA,122.74,1248,1290,388


In [5]:
df_rev.head()

,date,campaign_id,state,revenue
0,2023-07-24 0:00:00,64441550,GA,6370
1,2023-08-02 0:00:00,89363211,CA,6676
2,2023-08-03 0:00:00,86363015,CA,6239
3,2023-07-24 0:00:00,55304737,TX,6607
4,2023-08-17 0:00:00,86363015,NY,7014


## QUESTION 1: Provide a query to get the sum of impressions by day

In [26]:
query1 = """SELECT date, SUM(IMPRESSIONS) AS Total_Impressions_Per_Day FROM df_performance
           GROUP BY date 
           ORDER BY date ASC;"""
result_df1 = sqldf(query1, locals())

In [27]:
result_df1

,date,Total_Impressions_Per_Day
0,2023-07-24 0:00:00,1423
1,2023-07-26 0:00:00,1547
2,2023-07-27 0:00:00,2295
3,2023-07-28 0:00:00,8142
4,2023-07-29 0:00:00,3511
5,2023-07-30 0:00:00,3001
6,2023-08-02 0:00:00,1532
7,2023-08-03 0:00:00,3189
8,2023-08-04 0:00:00,3254
9,2023-08-05 0:00:00,1088


## QUESTION 2: Write a query to get the top three revenue-generating states in order of best to worst. How much revenue did the third best state generate?

In [11]:
query2 = """SELECT state, SUM(revenue) AS TOTAL_REV, MIN(date) AS MIN_DATE, MAX(date) AS MAX_DATE FROM df_rev
           GROUP BY state
           ORDER BY TOTAL_REV DESC
           LIMIT 3;"""
result_df2 = sqldf(query2, locals())

In [24]:
result_df2

,state,TOTAL_REV,MIN_DATE,MAX_DATE
0,NY,46398,2023-07-29 0:00:00,2023-08-17 0:00:00
1,GA,39666,2023-07-24 0:00:00,2023-08-13 0:00:00
2,OH,37577,2023-07-31 0:00:00,2023-08-11 0:00:00


In [13]:
## The 3rd best state (Ohio) generated $37,577
## Also outputted the date ranges since that might affect the amount of revenue generated 

## QUESTION 3: Write a query that shows total cost, impressions, clicks, and revenue of each campaign. Make sure to include the campaign name in the output.

In [22]:
query3 = """
           SELECT name AS CAMPAIGN_NAME,
                  SUM(cost) AS TOTAL_COST, 
                  SUM(impressions) AS TOTAL_IMPRESSIONS, 
                  SUM(clicks) AS TOTAL_CLICKS,
                  SUM(revenue) AS TOTAL_REV 
           FROM 
               (SELECT * 
                FROM df_performance t1 JOIN df_campaign t2 ON t1.campaign_id = t2.id) AS table_1_2
           JOIN df_rev ON table_1_2.campaign_id = df_rev.campaign_id
           GROUP BY name
           ORDER BY CAMPAIGN_NAME ASC
           ;"""
            
result_df3 = sqldf(query3, locals())

In [23]:
result_df3

,CAMPAIGN_NAME,TOTAL_COST,TOTAL_IMPRESSIONS,TOTAL_CLICKS,TOTAL_REV
0,Campaign1,4170.51,42810,38724,151792
1,Campaign2,4075.50,40938,29652,155308
2,Campaign3,15809.04,158280,116048,551672
3,Campaign4,3970.14,47508,33318,163396
4,Campaign5,4077.15,25641,33663,136404


## QUESTION 4: Write a query to get the number of conversions of Campaign5 by state. Which state generated the most conversions for this campaign?

In [35]:
query4 = """
           SELECT state, SUM(conversions) AS TOTAL_CONVERSIONS
           FROM (SELECT * 
                FROM df_performance t1 JOIN df_campaign t2 ON t1.campaign_id = t2.id) AS table_1_2
           JOIN df_rev ON table_1_2.campaign_id = df_rev.campaign_id
           WHERE name = 'Campaign5'
           GROUP BY state
           ORDER BY TOTAL_CONVERSIONS DESC
           ;"""
            
result_df4 = sqldf(query4, locals())

In [36]:
result_df4

,state,TOTAL_CONVERSIONS
0,GA,3342
1,NY,2228
2,OH,1114
3,CA,1114


In [37]:
## For Campaign5, Georgia generated the most amount of conversions (3342)

## QUESTION 5: In your opinion, which campaign was the most efficient, and why?

In [40]:
query5 = """
           SELECT name AS CAMPAIGN_NAME,
                  SUM(cost) AS TOTAL_COST, 
                  SUM(impressions) AS TOTAL_IMPRESSIONS, 
                  SUM(clicks) AS TOTAL_CLICKS,
                  SUM(revenue) AS TOTAL_REV,
                  SUM(revenue) - SUM(cost) AS PROFIT
           FROM 
               (SELECT * 
                FROM df_performance t1 JOIN df_campaign t2 ON t1.campaign_id = t2.id) AS table_1_2
           JOIN df_rev ON table_1_2.campaign_id = df_rev.campaign_id
           GROUP BY name
           ORDER BY PROFIT DESC
           ;"""
            
result_df5 = sqldf(query5, locals())

In [41]:
result_df5

,CAMPAIGN_NAME,TOTAL_COST,TOTAL_IMPRESSIONS,TOTAL_CLICKS,TOTAL_REV,PROFIT
0,Campaign3,15809.04,158280,116048,551672,535862.96
1,Campaign4,3970.14,47508,33318,163396,159425.86
2,Campaign2,4075.50,40938,29652,155308,151232.50
3,Campaign1,4170.51,42810,38724,151792,147621.49
4,Campaign5,4077.15,25641,33663,136404,132326.85


In [ ]:
## ANSWER --> CAMPAIGN 3
## When we calculate the profit for each campaign (revenue - cost), it is obvious that Campaign 3 did much better than the rest.

## QUESTION 6: Write a query that showcases the best day of the week (e.g., Sunday, Monday, Tuesday, etc.) to run ads.

In [89]:
query6 = """
           SELECT 
               DAY, 
               SUM(TOTAL_COST) AS TOTAL_COST, 
               SUM(TOTAL_IMPRESSIONS) AS TOTAL_IMPRESSIONS, 
               SUM(TOTAL_CLICKS) AS TOTAL_CLICKS,
               SUM(TOTAL_REV) AS TOTAL_REV,
               SUM(PROFIT) AS PROFIT 
           FROM
               (SELECT df_rev.date AS DATE,
                      CASE
                          WHEN df_rev.date in ('2023-07-24 0:00:00','2023-07-31 0:00:00', '2023-08-07 0:00:00', '2023-08-14 0:00:00', '2023-08-21 0:00:00') THEN 'MONDAY'
                          WHEN df_rev.date in ('2023-07-25 0:00:00','2023-08-01 0:00:00', '2023-08-08 0:00:00', '2023-08-15 0:00:00', '2023-08-22 0:00:00') THEN 'TUESDAY'
                          WHEN df_rev.date in ('2023-07-26 0:00:00','2023-08-02 0:00:00', '2023-08-09 0:00:00', '2023-08-16 0:00:00', '2023-08-23 0:00:00') THEN 'WEDNESDAY'
                          WHEN df_rev.date in ('2023-07-27 0:00:00','2023-08-03 0:00:00', '2023-08-10 0:00:00', '2023-08-17 0:00:00', '2023-08-24 0:00:00') THEN 'THURSDAY'
                          WHEN df_rev.date in ('2023-07-28 0:00:00','2023-08-04 0:00:00', '2023-08-11 0:00:00', '2023-08-18 0:00:00', '2023-08-25 0:00:00') THEN 'FRIDAY'
                          WHEN df_rev.date in ('2023-07-29 0:00:00','2023-08-05 0:00:00', '2023-08-12 0:00:00', '2023-08-19 0:00:00', '2023-08-26 0:00:00') THEN 'SATURDAY'
                          WHEN df_rev.date in ('2023-07-30 0:00:00','2023-08-06 0:00:00', '2023-08-13 0:00:00', '2023-08-20 0:00:00', '2023-08-27 0:00:00') THEN 'SUNDAY'
                      END AS DAY,
                      SUM(cost) AS TOTAL_COST, 
                      SUM(impressions) AS TOTAL_IMPRESSIONS, 
                      SUM(clicks) AS TOTAL_CLICKS,
                      SUM(revenue) AS TOTAL_REV,
                      SUM(revenue) - SUM(cost) AS PROFIT
               FROM 
                   (SELECT * 
                    FROM df_performance t1 JOIN df_campaign t2 ON t1.campaign_id = t2.id) AS table_1_2
               JOIN df_rev ON table_1_2.campaign_id = df_rev.campaign_id
               GROUP BY df_rev.date
               ORDER BY DATE DESC)
            GROUP BY DAY
            ORDER BY PROFIT DESC
           ;"""
            
result_df6 = sqldf(query6, locals())

In [90]:
result_df6

,DAY,TOTAL_COST,TOTAL_IMPRESSIONS,TOTAL_CLICKS,TOTAL_REV,PROFIT
0,MONDAY,9745.16,91437,75795,326322,316576.84
1,WEDNESDAY,6551.39,62945,52469,249250,242698.61
2,THURSDAY,5937.33,63324,45671,221492,215554.67
3,SATURDAY,5924.88,57503,46729,209933,204008.12
4,TUESDAY,1323.38,15836,11106,52916,51592.62
5,FRIDAY,1358.50,13646,9884,51152,49793.50
6,SUNDAY,1261.70,10486,9751,47507,46245.30


In [91]:
## ANSWER --> MONDAY
## Mondays tend to have the highest profit, impressions, clicks, and revenue out of all the other days of the week 
## Comment: I would have used DATE_TRUNC() or DATEPART() to obtain the number day and used a case statement to convert that to the day of the week by leveraging the modulus operator, but pandasql doesn't support that sql function.
##.         CASE WHEN DAY%7 = 3 THEN 'MONDAY'
##               WHEN DAY%7 = 4 THEN 'TUESDAY'
##               ...